## **Skript für das Scrapen der Twitterdaten**

In [ ]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook
from tweepy import OAuthHandler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/Manu Master/twitterapi.txt') as f:
  lines = f.read().splitlines()
  consumer_key = lines[0]
  consumer_secret = lines[1]
  access_key = lines[3]
  access_secret = lines[4]
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
#api = tweepy.API(auth)
api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

# Aufnahmefilter: Welche Hashtags berücksichtigt werden **sollen**

In [ ]:
search_words = "#covidvaccination OR #AstraZeneca OR #curevac OR #Moderna OR #Pfizer OR #BioNTech OR #covishield OR #covaxin OR #Sinopharm OR #SputnikV -filter:retweets"

#search_words = "#covid19 -filter:retweets"
date_since = "2021-04-01"
date_till = "2021-05-11"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since
              #until=date_till
              ).items(900)

In [ ]:
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

900it [00:36, 24.69it/s]


In [ ]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 900


# Die Spalten, die übernommen werden sollen

In [ ]:
from tqdm import tqdm, notebook
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 900/900 [04:33<00:00,  3.29it/s]


In [ ]:
#@title Standardtext für Titel
import pandas as pd
#f = '/content/drive/My Drive/Manu Master/twitter_data_analysis.csv'
#tweets_df.to_csv(f)
tweets_old_df = pd.read_csv("/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv")
print(f"past tweets: {tweets_old_df.shape}")
fileold = '/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'
if(os.path.exists(fileold) and os.path.isfile(fileold)):
  os.remove(fileold)
  print("file deleted")
else:
  print("file not found")

past tweets: (209345, 9)
file deleted


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:

tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 900 past tweets: 209345 all tweets: 210245


In [ ]:
tweets_all_df.drop_duplicates(subset = ["user_name", "text"], keep= 'first', inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (210239, 9)


In [ ]:
import os
# first check whether file exists or not
fl = '/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv'
tweets_all_df.to_csv(fl,index=False) 
# first check whether file exists or not
# calling remove method to delete the csv file
# in remove method you need to pass file name and type
file = '/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
  print("file deleted")
else:
  print("file not found")

In [ ]:
import shutil
original = r'/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv'
target = r'/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'

shutil.copyfile(original, target)


'/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'

In [ ]:
tweets_saved_df = pd.read_csv("/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv")
print(f"Last saved tweets: {tweets_saved_df.shape}")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Last saved tweets: (210239, 9)


**Bereinigung der Zeilendaten von Twitter: (i) Auswahl nur der Tweets mit Benutzerort, user_created und Tweet-Text, um mehrdeutige Daten zu vermeiden; (ii) Entfernen von nicht ascii-Zeichen aus dem Twitter-Text (iii) Entfernen doppelter Daten in username, user_created oder text und Zurücksetzen des Index**

In [ ]:
import numpy as np
df_new=tweets_saved_df[tweets_saved_df["user_location"].notnull()] 
df_ultranew=df_new[df_new['user_location'].map(lambda x: x.isascii())]

df_new=df_ultranew[df_ultranew["text"].notnull()] 
#df_ultranew=df_new[df_new['text'].map(lambda x: x.isascii())]
modified_df = df_ultranew.dropna(subset=['user_location'])
modified_df['user_created'].replace('', np.nan, inplace=True)
modified_df = df_ultranew.dropna(subset=['user_created'])
modified_df = modified_df.drop_duplicates(subset=['user_name','user_created','text'])
modified_df= modified_df.reset_index(drop=True)

In [ ]:
import os
fl = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetsI.csv'
modified_df.to_csv(fl,index=False) 
file = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetsbackupI.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
  print("file deleted")
else:
  print("file not found")

file deleted


In [ ]:
import shutil
original = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetsI.csv'
target = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetsbackupI.csv'
shutil.copyfile(original, target)
print("file copied and backed up")

file copied and backed up


In [ ]:
modified_df[-50:]

***konsolidieren mit Kaggle Daten***


In [ ]:
tweets_frominternet_df = pd.read_csv("/content/drive/My Drive/Manu Master/dataset/vaccination_all_tweets.csv")
#tweets_frominternet_df = pd.read_csv("D://vaccination_all_tweets.csv")
tweets_frominternet_df = tweets_frominternet_df.drop(['source','retweets','favorites'], axis = 1)
print(f"Tweets from Internet: {tweets_frominternet_df.shape}")
tweets_frominternet_df[-5:]

In [ ]:
import numpy as np
df_newint=tweets_frominternet_df[tweets_frominternet_df["user_location"].notnull()] 
df_ultranewint=df_newint[df_newint['user_location'].map(lambda x: x.isascii())]

df_newint=df_ultranewint[df_ultranewint["text"].notnull()] 
#df_ultranew=df_new[df_new['text'].map(lambda x: x.isascii())]
modified_dfint = df_ultranewint.dropna(subset=['user_location'])
modified_dfint['user_created'].replace('', np.nan, inplace=True)
modified_dfint = df_ultranewint.dropna(subset=['user_created'])
modified_dfint = modified_dfint.drop_duplicates(subset=['user_name','user_created','text'])
modified_dfint= modified_dfint.reset_index(drop=True)
modified_dfint.drop(['id', 'user_followers','user_friends','user_favourites'], axis = 1,inplace=True)

In [ ]:
modified_dfint[:5]

saving data for later use with backup

In [ ]:
import os
fl = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetsprada.csv'
modified_dfint.to_csv(fl,index=False) 
file = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetspradabackup.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
  print("file saved/deleted and backedup")
else:
  print("file not found")
original = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetsprada.csv'
target = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetspradabackup.csv'
shutil.copyfile(original, target)



**Bitte TweetsearchII anschauen**